In [1]:
import grequests
import pandas as pd
import numpy as np
import src.Engines.gdc_engine as gdc_engine
import os
from importlib import reload
from flatten_json import flatten
import multiprocessing
import requests
import io
import src.Connectors.gdc_endpt_base as gdc_endpt_base
import concurrent.futures
import requests
import json
import src.Connectors.mongo_db_conn as mongodbc
from tqdm import tqdm
from google.cloud import bigquery
reload(mongodbc)
reload(gdc_endpt_base)
reload(gdc_engine)


<module 'src.Engines.gdc_engine' from '/Users/abhilashdhal/Projects/src/Engines/gdc_engine.py'>

## Load the cohorts

In [2]:
!pwd

/Users/abhilashdhal/Projects


In [3]:
all_meta = pd.read_csv('./de_gsea_data/metadata/all_metadata_rna_seq.csv')
lung_rna_seq_data = pd.read_csv('./de_gsea_data/rna_seq_data/lung_unstr_tumor_normal.csv')

In [15]:
all_meta

,Unnamed: 0,id,case_id,alcohol_history,years_smoked,tissue_or_organ_of_origin,days_to_last_follow_up,age_at_diagnosis,primary_diagnosis,primary_site,...,tissue_type,sample_type,race,gender,ethnicity,file_name,file_id,data_type,workflow_type,experimental_strategy
0,0,26e28409-2493-4bd8-86f5-dc7b85846081,9097285b-eeb1-4f36-a0ab-0d0592043792,NaN,NaN,"Esophagus, NOS",NaN,28226.0,"Adenocarcinoma, NOS",NaN,...,Tumor,Next Generation Cancer Model,white,male,not hispanic or latino,b231e28d-7a0f-41ea-864c-69accc514428.rna_seq.a...,26e28409-2493-4bd8-86f5-dc7b85846081,Gene Expression Quantification,STAR - Counts,RNA-Seq
1,1,6abbf9ed-e3bd-489e-a67f-2624d427c81a,653a556f-c127-421d-8118-4bf98929fe53,NaN,NaN,Not Reported,NaN,NaN,"Intraductal papillary-mucinous carcinoma, inva...",NaN,...,Tumor,Primary Tumor,not reported,male,not reported,70f93c7b-27fa-47e4-a602-106bc35732d6.rna_seq.a...,6abbf9ed-e3bd-489e-a67f-2624d427c81a,Gene Expression Quantification,STAR - Counts,RNA-Seq
2,2,1b378dce-6ab0-436a-95cd-0a70774c4766,10e69494-3ce2-4c13-ab1b-6fc0e6b14853,NaN,NaN,"Esophagus, NOS",NaN,NaN,"Adenocarcinoma, NOS",NaN,...,Tumor,Next Generation Cancer Model,Unknown,male,Unknown,b6e57b66-57fa-4a30-941e-86a5686ef4c4.rna_seq.a...,1b378dce-6ab0-436a-95cd-0a70774c4766,Gene Expression Quantification,STAR - Counts,RNA-Seq
3,3,842f9343-e55f-4db8-aa45-7540b1bc9f1c,653a556f-c127-421d-8118-4bf98929fe53,NaN,NaN,Not Reported,NaN,NaN,"Intraductal papillary-mucinous carcinoma, inva...",NaN,...,Tumor,Metastatic,not reported,male,not reported,1bf7dccb-5d75-4ab8-951f-6ee263aa1b48.rna_seq.a...,842f9343-e55f-4db8-aa45-7540b1bc9f1c,Gene Expression Quantification,STAR - Counts,RNA-Seq
4,4,1a2bf971-f722-403d-aa30-4882c164a300,8d169383-a8af-4450-a837-01da35fb72c5,NaN,NaN,"Rectum, NOS",NaN,20434.0,Mucinous adenocarcinoma,NaN,...,Tumor,Primary Tumor,white,male,not hispanic or latino,7836ae1b-35ea-4273-b9ed-2bb72f1b3d06.rna_seq.a...,1a2bf971-f722-403d-aa30-4882c164a300,Gene Expression Quantification,STAR - Counts,RNA-Seq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24806,24806,16a1869c-00cf-4041-ac21-774e03eef72e,307c905f-c276-4501-988a-b083f9462a98,NaN,NaN,"Bone, NOS",NaN,4997.0,Ewing sarcoma,NaN,...,Tumor,Metastatic,white,male,Unknown,245c580b-ac5e-483f-9910-209360d54923.rna_seq.a...,16a1869c-00cf-4041-ac21-774e03eef72e,Gene Expression Quantification,STAR - Counts,RNA-Seq
24807,24807,b4d896c1-a4e8-49af-a3cb-530678170e7a,9f4d0076-c4a2-46d4-879f-896946338c09,NaN,NaN,"Skin, NOS",NaN,28529.0,Spindle cell sarcoma,NaN,...,Tumor,Metastatic,white,male,not hispanic or latino,2719f43f-11fc-487b-88bf-2848cccea68f.rna_seq.a...,b4d896c1-a4e8-49af-a3cb-530678170e7a,Gene Expression Quantification,STAR - Counts,RNA-Seq
24808,24808,ff89537e-83d8-4977-99c8-fa803050a393,9f4d0076-c4a2-46d4-879f-896946338c09,NaN,NaN,"Skin, NOS",NaN,28529.0,Spindle cell sarcoma,NaN,...,Tumor,Next Generation Cancer Model,white,male,not hispanic or latino,cd351af3-cfe0-4f0e-a378-19eb448e0f3f.rna_seq.a...,ff89537e-83d8-4977-99c8-fa803050a393,Gene Expression Quantification,STAR - Counts,RNA-Seq
24809,24809,eca78805-8b1d-49b7-ad56-c68c53f25e67,66f2f8d4-f8e2-4bb6-93e1-3edd7c5a3b48,NaN,NaN,"Breast, NOS",NaN,30468.0,"Infiltrating duct carcinoma, NOS",NaN,...,Tumor,Metastatic,black or african american,female,not hispanic or latino,5b50264e-42a6-4fb0-836d-36ee9f11ec09.rna_seq.a...,eca78805-8b1d-49b7-ad56-c68c53f25e67,Gene Expression Quantification,STAR - Counts,RNA-Seq


In [4]:
all_meta[all_meta['primary_site'] == 'Blood']['sample_type'].value_counts()

sample_type
Primary Blood Derived Cancer - Bone Marrow                 2276
Recurrent Blood Derived Cancer - Bone Marrow                458
Primary Blood Derived Cancer - Peripheral Blood             418
Bone Marrow Normal                                          324
Recurrent Blood Derived Cancer - Peripheral Blood            31
Blood Derived Normal                                         31
Blood Derived Cancer - Bone Marrow, Post-treatment           25
Blood Derived Cancer - Peripheral Blood, Post-treatment       1
Name: count, dtype: int64

In [5]:
lung_rna_seq_data

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AC136977.1,AC078856.1,AC008763.4,AL592295.6,AC006486.3,AL391628.1,AP006621.6,file_id,tissue_type,case_id
0,4103,0,1692,1538,1274,1975,3179,4275,8109,1843,...,0,1,1,256,0,4,30,45c6656d-a8f3-4968-aa40-142f3a340dde,Tumor,fd9ee494-65fe-4de4-adff-7952a059b17f
1,2065,0,5496,195,267,323,3262,3443,2889,566,...,0,0,0,37,0,1,8,fb103781-9cee-4e30-a864-3d4e9c5b32ae,Tumor,49f975ea-a1cf-4a9b-bf13-030c68cc99e4
2,2280,1,2108,1569,1677,1065,1143,1329,2205,3267,...,0,0,1,177,0,8,2,5d703777-b3db-4ed5-952b-203a5641767e,Tumor,99599b60-4f5c-456b-8755-371b1aa7074e
3,2359,1,1815,833,329,3399,5915,2295,9746,2374,...,0,0,1,222,0,9,38,09152b74-f690-4dbc-a398-1f8dde28ec06,Tumor,f3e67161-1532-4b50-a9b5-0da80410c4d7
4,5355,4,2563,800,895,878,1708,3051,10762,1629,...,0,0,0,230,0,6,36,7a9de79e-7eaf-41f3-b7fd-bd59cbf64443,Tumor,bece6b8e-5d6c-4dd6-85a3-9b3a9c670aa7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,3418,0,2937,1008,592,1926,7334,4741,39350,3881,...,0,0,0,421,0,9,21,08d83462-5fcb-40c6-9d44-dbc42faa4df6,Tumor,cd902d08-215e-4bd0-88e4-4fd01ab43cbf
1146,667,10,554,716,619,455,2567,588,2194,738,...,0,15,0,228,0,1,0,1ab1279d-7ccd-4450-8de8-9987bf6627dd,Tumor,ccda26c1-a6d6-4317-8cf8-8a87e15ce12e
1147,3365,0,2244,777,451,507,2252,1525,2684,2750,...,0,0,2,242,0,10,27,7abd9747-a947-4b15-b51c-c82cda9f065c,Tumor,ceead734-1ce0-4385-b65a-a9c853b7308e
1148,3249,0,1969,1345,427,2713,4163,2441,1561,2614,...,0,0,0,570,0,20,21,f8670621-43a9-491c-81b7-8999ca9975a7,Tumor,d3ff328c-2488-47bd-8663-366b6812c7b5


In [6]:
gene_name_cols = lung_rna_seq_data.columns.to_numpy()[:60660]

In [12]:
all_cols = sorted(list(gene_name_cols))
all_cols.append('case_id')
all_cols.append('file_id')
lung_rna_seq_data_unq = lung_rna_seq_data.drop_duplicates(['case_id']).reset_index(drop=True)[all_cols]
lung_rna_seq_data_unq['expr_unstr_count'] = lung_rna_seq_data_unq[np.sort(gene_name_cols)].agg(list, axis=1)


In [14]:
lung_rna_seq_data_unq

,5S_rRNA,5S_rRNA.1,5S_rRNA.2,5S_rRNA.3,5S_rRNA.4,5S_rRNA.5,5S_rRNA.6,5S_rRNA.7,5S_rRNA.8,5_8S_rRNA,...,ZYX,ZYXP1,ZZEF1,ZZZ3,hsa-mir-1253,hsa-mir-423,snoZ196,case_id,file_id,expr_unstr_count
0,0,1,0,0,0,0,0,0,0,0,...,14677,0,2955,2204,0,0,0,fd9ee494-65fe-4de4-adff-7952a059b17f,45c6656d-a8f3-4968-aa40-142f3a340dde,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,0,0,0,0,0,0,0,0,0,...,3045,0,338,987,0,0,1,49f975ea-a1cf-4a9b-bf13-030c68cc99e4,fb103781-9cee-4e30-a864-3d4e9c5b32ae,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,4,0,0,0,0,0,0,0,0,...,7396,0,2149,2615,0,0,1,99599b60-4f5c-456b-8755-371b1aa7074e,5d703777-b3db-4ed5-952b-203a5641767e,"[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0,0,0,0,0,0,0,0,0,0,...,19516,0,4913,2474,0,0,5,f3e67161-1532-4b50-a9b5-0da80410c4d7,09152b74-f690-4dbc-a398-1f8dde28ec06,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,2,0,0,0,0,0,0,0,0,...,11430,0,1967,1556,0,0,3,bece6b8e-5d6c-4dd6-85a3-9b3a9c670aa7,7a9de79e-7eaf-41f3-b7fd-bd59cbf64443,"[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,0,3,0,1,0,0,0,0,0,0,...,5410,0,11677,4551,0,0,3,cc68632c-b1e3-491b-b562-9468e2d1c101,db0caa6a-e863-4dad-a911-0c69127be9df,"[0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1012,0,2,0,0,0,0,0,0,0,0,...,13421,0,5345,5134,0,0,0,cd902d08-215e-4bd0-88e4-4fd01ab43cbf,08d83462-5fcb-40c6-9d44-dbc42faa4df6,"[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1013,0,1,0,0,0,1,0,0,0,0,...,8414,0,1583,1651,0,0,1,ceead734-1ce0-4385-b65a-a9c853b7308e,7abd9747-a947-4b15-b51c-c82cda9f065c,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1014,0,1,0,0,0,2,0,0,0,0,...,15334,0,4909,2985,0,0,0,d3ff328c-2488-47bd-8663-366b6812c7b5,f8670621-43a9-491c-81b7-8999ca9975a7,"[0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [16]:
data_for_bq = lung_rna_seq_data_unq[['case_id', 'file_id', 'expr_unstr_count']]
data_bq_with_labels = pd.merge(data_for_bq, all_meta[['file_id', 'case_id', 'tissue_type', 'sample_type', 'primary_site', 'tissue_or_organ_of_origin', 'age_at_diagnosis', 'primary_diagnosis', 'race', 'gender']], on=['file_id', 'case_id'])

In [17]:
data_bq_with_labels

,case_id,file_id,expr_unstr_count,tissue_type,sample_type,primary_site,tissue_or_organ_of_origin,age_at_diagnosis,primary_diagnosis,race,gender
0,fd9ee494-65fe-4de4-adff-7952a059b17f,45c6656d-a8f3-4968-aa40-142f3a340dde,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Lower lobe, lung",22800.0,"Squamous cell carcinoma, NOS",white,male
1,49f975ea-a1cf-4a9b-bf13-030c68cc99e4,fb103781-9cee-4e30-a864-3d4e9c5b32ae,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Lung, NOS",22754.0,"Squamous cell carcinoma, NOS",white,female
2,99599b60-4f5c-456b-8755-371b1aa7074e,5d703777-b3db-4ed5-952b-203a5641767e,"[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Lower lobe, lung",29827.0,"Squamous cell carcinoma, NOS",not reported,male
3,f3e67161-1532-4b50-a9b5-0da80410c4d7,09152b74-f690-4dbc-a398-1f8dde28ec06,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Upper lobe, lung",18990.0,"Squamous cell carcinoma, NOS",white,male
4,bece6b8e-5d6c-4dd6-85a3-9b3a9c670aa7,7a9de79e-7eaf-41f3-b7fd-bd59cbf64443,"[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Lower lobe, lung",27749.0,"Squamous cell carcinoma, NOS",white,male
...,...,...,...,...,...,...,...,...,...,...,...
1011,cc68632c-b1e3-491b-b562-9468e2d1c101,db0caa6a-e863-4dad-a911-0c69127be9df,"[0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Middle lobe, lung",25791.0,"Adenocarcinoma, NOS",white,male
1012,cd902d08-215e-4bd0-88e4-4fd01ab43cbf,08d83462-5fcb-40c6-9d44-dbc42faa4df6,"[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Upper lobe, lung",21214.0,"Bronchiolo-alveolar adenocarcinoma, NOS",white,male
1013,ceead734-1ce0-4385-b65a-a9c853b7308e,7abd9747-a947-4b15-b51c-c82cda9f065c,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Upper lobe, lung",20888.0,"Adenocarcinoma, NOS",white,male
1014,d3ff328c-2488-47bd-8663-366b6812c7b5,f8670621-43a9-491c-81b7-8999ca9975a7,"[0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Tumor,Primary Tumor,Lung,"Upper lobe, lung",28024.0,"Adenocarcinoma, NOS",white,female


In [18]:
data_bq_with_labels['cohort'] = 'Lung_RNA_Seq'

## 0.2. What are the different primary diagnosis categories for Lung Tumor and Normal RNA-Seq Samples?

In [21]:
data_bq_with_labels[data_bq_with_labels['tissue_type'] == 'Normal']['primary_diagnosis'].value_counts()

primary_diagnosis
Adenocarcinoma, NOS                                     22
Squamous cell carcinoma, NOS                            20
Mucinous adenocarcinoma                                  4
Squamous cell carcinoma, keratinizing, NOS               2
Basaloid squamous cell carcinoma                         2
Adenocarcinoma with mixed subtypes                       2
Squamous cell carcinoma, small cell, nonkeratinizing     1
Bronchiolo-alveolar carcinoma, non-mucinous              1
Name: count, dtype: int64

In [22]:
data_bq_with_labels[data_bq_with_labels['tissue_type'] == 'Tumor']['primary_diagnosis'].value_counts()

primary_diagnosis
Squamous cell carcinoma, NOS                                 446
Adenocarcinoma, NOS                                          289
Adenocarcinoma with mixed subtypes                           106
Papillary adenocarcinoma, NOS                                 22
Acinar cell carcinoma                                         22
Bronchiolo-alveolar carcinoma, non-mucinous                   18
Basaloid squamous cell carcinoma                              12
Squamous cell carcinoma, keratinizing, NOS                    11
Mucinous adenocarcinoma                                        9
Solid carcinoma, NOS                                           6
Bronchio-alveolar carcinoma, mucinous                          5
Papillary squamous cell carcinoma                              4
Micropapillary carcinoma, NOS                                  3
Bronchiolo-alveolar adenocarcinoma, NOS                        3
Squamous cell carcinoma, large cell, nonkeratinizing, NOS      3
Clear c

In [11]:
data_bq_with_labels.to_json('lung_cancer_cohort.json', orient='records', lines=True)

## Up-load to bigquery

In [19]:
### Convert dataframe to JSON object
json_data = data_bq_with_labels.to_json(orient = 'records')
json_object = json.loads(json_data)

client = bigquery.Client(project='rnaseqml')
table_id = 'rnaseqml.rnaseqexpression.expr'
schema = [
    bigquery.SchemaField("case_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("file_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("expr_unstr_count","INTEGER",mode="REPEATED",
    ),

    bigquery.SchemaField("tissue_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("sample_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("primary_site", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tissue_or_organ_of_origin", "STRING", mode="NULLABLE"),        
    bigquery.SchemaField("age_at_diagnosis", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("primary_diagnosis", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("race", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("gender", "STRING", mode="NULLABLE"),
                    bigquery.SchemaField("cohort", "STRING", mode="NULLABLE")
    ]
table = bigquery.Table(table_id, schema=schema)

# API request
# # job = client.load_table_from_dataframe(
# expr_meta, table_id_meta, job_config=job_config
# )

In [20]:
job_config = bigquery.LoadJobConfig(
source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON, schema=schema
)

job = client.load_table_from_json(json_object, table_id, job_config = job_config)

## Create clustering for tables

In [21]:
from google.cloud import bigquery

client = bigquery.Client(project='rnaseqml')
table_id = 'rnaseqml.rnaseqexpression.expr'
schema = [
    bigquery.SchemaField("case_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("file_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("expr_unstr_count","INTEGER",mode="REPEATED",
    ),

    bigquery.SchemaField("tissue_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("sample_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("primary_site", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tissue_or_organ_of_origin", "STRING", mode="NULLABLE"),        
    bigquery.SchemaField("age_at_diagnosis", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("primary_diagnosis", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("race", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("gender", "STRING", mode="NULLABLE"),
                    bigquery.SchemaField("cohort", "STRING", mode="NULLABLE")
    ]
table = bigquery.Table(table_id, schema=schema)
table.clustering_fields = ["primary_site", "tissue_type", 'primary_diagnosis']
table = client.create_table(table)  # Make an API request.
print(
    "Created clustered table {}.{}.{}".format(
        table.project, table.dataset_id, table.table_id
    )
)

python(50962) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/rnaseqml/datasets/rnaseqexpression/tables?prettyPrint=false: Already Exists: Table rnaseqml:rnaseqexpression.expr

## Check number of array values for bigquery upload

In [1]:
import grequests
import pandas as pd
import numpy as np
import src.Engines.gdc_engine as gdc_engine
import os
from importlib import reload
from flatten_json import flatten
from tqdm import tqdm 
import requests
import json
from google.cloud import bigquery
import hashlib
reload(gdc_engine)

<module 'src.Engines.gdc_engine' from '/Users/abhilashdhal/Projects/src/Engines/gdc_engine.py'>

In [2]:
params = {
'files.experimental_strategy': 'RNA-Seq', 
'data_type': 'Gene Expression Quantification'
}

gdc_eng_inst = gdc_engine.GDCEngine(**params)
cohort_metadata = gdc_eng_inst._get_rna_seq_metadata()
cohort_metadata = cohort_metadata['metadata']

dict_keys(['files.experimental_strategy', 'data_type'])
file_id,file_name,experimental_strategy,data_type,platform,cases.case_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.primary_diagnosis,cases.diagnoses.tumor_stage,cases.diagnoses.tumor_grade,cases.diagnoses.treatments.treatment_or_therapy,cases.diagnoses.days_to_last_follow_up,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_death,cases.project.primary_site,analysis.workflow_type,cases.demographic.ethnicity,cases.demographic.gender,cases.demographic.race,cases.diagnoses.tissue_or_organ_of_origin,cases.exposures.bmi,cases.exposures.alcohol_history,cases.exposures.years_smoked,cases.samples.tissue_type,cases.samples.sample_type


In [16]:
cohort_metadata 

,id,case_id,alcohol_history,years_smoked,tissue_or_organ_of_origin,days_to_last_follow_up,age_at_diagnosis,primary_diagnosis,primary_site,tumor_grade,...,tissue_type,sample_type,race,gender,ethnicity,file_name,file_id,data_type,workflow_type,experimental_strategy
0,26e28409-2493-4bd8-86f5-dc7b85846081,9097285b-eeb1-4f36-a0ab-0d0592043792,None,NaN,"Esophagus, NOS",NaN,28226.0,"Adenocarcinoma, NOS",None,None,...,Tumor,Next Generation Cancer Model,white,male,not hispanic or latino,b231e28d-7a0f-41ea-864c-69accc514428.rna_seq.a...,26e28409-2493-4bd8-86f5-dc7b85846081,Gene Expression Quantification,STAR - Counts,RNA-Seq
1,6abbf9ed-e3bd-489e-a67f-2624d427c81a,653a556f-c127-421d-8118-4bf98929fe53,None,NaN,Not Reported,NaN,NaN,"Intraductal papillary-mucinous carcinoma, inva...",None,None,...,Tumor,Primary Tumor,not reported,male,not reported,70f93c7b-27fa-47e4-a602-106bc35732d6.rna_seq.a...,6abbf9ed-e3bd-489e-a67f-2624d427c81a,Gene Expression Quantification,STAR - Counts,RNA-Seq
2,1b378dce-6ab0-436a-95cd-0a70774c4766,10e69494-3ce2-4c13-ab1b-6fc0e6b14853,None,NaN,"Esophagus, NOS",NaN,NaN,"Adenocarcinoma, NOS",None,G3,...,Tumor,Next Generation Cancer Model,Unknown,male,Unknown,b6e57b66-57fa-4a30-941e-86a5686ef4c4.rna_seq.a...,1b378dce-6ab0-436a-95cd-0a70774c4766,Gene Expression Quantification,STAR - Counts,RNA-Seq
3,842f9343-e55f-4db8-aa45-7540b1bc9f1c,653a556f-c127-421d-8118-4bf98929fe53,None,NaN,Not Reported,NaN,NaN,"Intraductal papillary-mucinous carcinoma, inva...",None,None,...,Tumor,Metastatic,not reported,male,not reported,1bf7dccb-5d75-4ab8-951f-6ee263aa1b48.rna_seq.a...,842f9343-e55f-4db8-aa45-7540b1bc9f1c,Gene Expression Quantification,STAR - Counts,RNA-Seq
4,1a2bf971-f722-403d-aa30-4882c164a300,8d169383-a8af-4450-a837-01da35fb72c5,None,NaN,"Rectum, NOS",NaN,20434.0,Mucinous adenocarcinoma,None,GX,...,Tumor,Primary Tumor,white,male,not hispanic or latino,7836ae1b-35ea-4273-b9ed-2bb72f1b3d06.rna_seq.a...,1a2bf971-f722-403d-aa30-4882c164a300,Gene Expression Quantification,STAR - Counts,RNA-Seq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24806,16a1869c-00cf-4041-ac21-774e03eef72e,307c905f-c276-4501-988a-b083f9462a98,None,NaN,"Bone, NOS",NaN,4997.0,Ewing sarcoma,None,None,...,Tumor,Metastatic,white,male,Unknown,245c580b-ac5e-483f-9910-209360d54923.rna_seq.a...,16a1869c-00cf-4041-ac21-774e03eef72e,Gene Expression Quantification,STAR - Counts,RNA-Seq
24807,b4d896c1-a4e8-49af-a3cb-530678170e7a,9f4d0076-c4a2-46d4-879f-896946338c09,None,NaN,"Skin, NOS",NaN,28529.0,Spindle cell sarcoma,None,None,...,Tumor,Metastatic,white,male,not hispanic or latino,2719f43f-11fc-487b-88bf-2848cccea68f.rna_seq.a...,b4d896c1-a4e8-49af-a3cb-530678170e7a,Gene Expression Quantification,STAR - Counts,RNA-Seq
24808,ff89537e-83d8-4977-99c8-fa803050a393,9f4d0076-c4a2-46d4-879f-896946338c09,None,NaN,"Skin, NOS",NaN,28529.0,Spindle cell sarcoma,None,None,...,Tumor,Next Generation Cancer Model,white,male,not hispanic or latino,cd351af3-cfe0-4f0e-a378-19eb448e0f3f.rna_seq.a...,ff89537e-83d8-4977-99c8-fa803050a393,Gene Expression Quantification,STAR - Counts,RNA-Seq
24809,eca78805-8b1d-49b7-ad56-c68c53f25e67,66f2f8d4-f8e2-4bb6-93e1-3edd7c5a3b48,None,NaN,"Breast, NOS",NaN,30468.0,"Infiltrating duct carcinoma, NOS",None,G3,...,Tumor,Metastatic,black or african american,female,not hispanic or latino,5b50264e-42a6-4fb0-836d-36ee9f11ec09.rna_seq.a...,eca78805-8b1d-49b7-ad56-c68c53f25e67,Gene Expression Quantification,STAR - Counts,RNA-Seq


In [3]:
363*60660

22019580

In [4]:
241699920/22019580

10.9765908341576

In [5]:
site='Stomach'
df = gdc_eng_inst.run_rna_seq_data_matrix_creation(primary_site=site,downstream_analysis='DE')
df = df.set_index('file_id')
df = df.reset_index()

dict_keys(['files.experimental_strategy', 'data_type'])
file_id,file_name,experimental_strategy,data_type,platform,cases.case_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.primary_diagnosis,cases.diagnoses.tumor_stage,cases.diagnoses.tumor_grade,cases.diagnoses.treatments.treatment_or_therapy,cases.diagnoses.days_to_last_follow_up,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_death,cases.project.primary_site,analysis.workflow_type,cases.demographic.ethnicity,cases.demographic.gender,cases.demographic.race,cases.diagnoses.tissue_or_organ_of_origin,cases.exposures.bmi,cases.exposures.alcohol_history,cases.exposures.years_smoked,cases.samples.tissue_type,cases.samples.sample_type


100%|██████████| 8/8 [03:03<00:00, 22.94s/it]


In [11]:
sub_df = df.iloc[:5, :]

In [12]:
sub_df

,file_id,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,AC078856.1,AC008763.4,AL592295.6,AC006486.3,AL391628.1,AP006621.6,tissue_type,sample_type,primary_site,case_id
0,a990aca5-8d28-471d-9b18-0422d6fafe25,1566,4,1268,450,141,444,4936,2068,1510,...,0,0,126,0,5,28,Tumor,Primary Tumor,Stomach,05ce16d4-a999-491a-91ad-9449a57228ff
1,84241123-0f4c-40de-b0b4-b9966af2c101,1020,3,724,393,64,203,1996,1609,1511,...,0,0,79,0,4,12,Normal,Solid Tissue Normal,Stomach,05ce16d4-a999-491a-91ad-9449a57228ff
2,b8275137-3052-4f60-b11d-f15499863648,7359,2,6326,845,1159,171,770,5009,5923,...,1,0,159,0,3,34,Tumor,Primary Tumor,Stomach,018f93ad-92ef-4bbb-a0fb-dc407a289daa
3,0390dab8-28c2-466f-8d58-07870b2f0a49,1818,1,2483,1091,454,615,7670,3309,1907,...,2,0,445,0,17,22,Tumor,Primary Tumor,Stomach,0a11e958-7ec0-493c-bd7a-eb17536facd7
4,7ca5701e-8e7e-4238-8225-7928a4c15e18,1918,0,5849,1540,748,1395,4240,6175,3557,...,5,0,425,0,6,77,Tumor,Primary Tumor,Stomach,00fdf96d-c48a-47e0-b91d-576c52e6819c


In [13]:
gene_cols = sub_df.columns.to_numpy()[1:60661]
# sub_df['all_values'] = sub_df[gene_cols].apply(lambda row: row.tolist(), axis=1)
sub_df['expr_unstr_count'] = [list(row) for row in  sub_df[gene_cols].values]

/var/folders/ng/bwk7d4ds7wz95l011dbvtc9r0000gn/T/ipykernel_72585/2217719600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['expr_unstr_count'] = [list(row) for row in  sub_df[gene_cols].values]


In [14]:
sub_df

,file_id,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,AC008763.4,AL592295.6,AC006486.3,AL391628.1,AP006621.6,tissue_type,sample_type,primary_site,case_id,expr_unstr_count
0,a990aca5-8d28-471d-9b18-0422d6fafe25,1566,4,1268,450,141,444,4936,2068,1510,...,0,126,0,5,28,Tumor,Primary Tumor,Stomach,05ce16d4-a999-491a-91ad-9449a57228ff,"[1566, 4, 1268, 450, 141, 444, 4936, 2068, 151..."
1,84241123-0f4c-40de-b0b4-b9966af2c101,1020,3,724,393,64,203,1996,1609,1511,...,0,79,0,4,12,Normal,Solid Tissue Normal,Stomach,05ce16d4-a999-491a-91ad-9449a57228ff,"[1020, 3, 724, 393, 64, 203, 1996, 1609, 1511,..."
2,b8275137-3052-4f60-b11d-f15499863648,7359,2,6326,845,1159,171,770,5009,5923,...,0,159,0,3,34,Tumor,Primary Tumor,Stomach,018f93ad-92ef-4bbb-a0fb-dc407a289daa,"[7359, 2, 6326, 845, 1159, 171, 770, 5009, 592..."
3,0390dab8-28c2-466f-8d58-07870b2f0a49,1818,1,2483,1091,454,615,7670,3309,1907,...,0,445,0,17,22,Tumor,Primary Tumor,Stomach,0a11e958-7ec0-493c-bd7a-eb17536facd7,"[1818, 1, 2483, 1091, 454, 615, 7670, 3309, 19..."
4,7ca5701e-8e7e-4238-8225-7928a4c15e18,1918,0,5849,1540,748,1395,4240,6175,3557,...,0,425,0,6,77,Tumor,Primary Tumor,Stomach,00fdf96d-c48a-47e0-b91d-576c52e6819c,"[1918, 0, 5849, 1540, 748, 1395, 4240, 6175, 3..."


In [18]:
def create_identifier(row, existing_identifiers):
    identifier_str = f"{row['primary_site']}_{row['tissue_type']}_{row['primary_diagnosis']}"
    identifier = int(hashlib.md5(identifier_str.encode()).hexdigest(), 16) % (10 ** 8)
    
    while identifier in existing_identifiers:
        # Create a new identifier if there is a clash
        identifier_str += "_duplicate"
        identifier = int(hashlib.md5(identifier_str.encode()).hexdigest(), 16) % (10 ** 8)
    
    existing_identifiers.add(identifier)
    return identifier

data_for_bq = sub_df[['case_id', 'file_id', 'expr_unstr_count', 'tissue_type', 'sample_type', 'primary_site']]
data_bq_with_labels = pd.merge(data_for_bq, cohort_metadata[['file_id', 'case_id', 'tissue_or_organ_of_origin', 'age_at_diagnosis', 'primary_diagnosis', 'race', 'gender']], on=['file_id', 'case_id'])
# data_bq_with_labels['cohort'] = site
data_bq_with_labels['group_identifier'] = data_bq_with_labels.apply(create_identifier, axis=1)

TypeError: create_identifier() missing 1 required positional argument: 'existing_identifiers'